In [1]:
""" This module prepares midi file data and feeds it to the neural
    network for training """
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.layers import Dense, Activation, Flatten

def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in glob.glob("D:/Study/NEU/Neural Network and AI/final/Classical-Piano-Composer-master/Classical-Piano-Composer-master/midi_songs/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('D:/Study/NEU/Neural Network and AI/final/Classical-Piano-Composer-master/Classical-Piano-Composer-master/data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.1))
    model.add(LSTM(512, return_sequences=True))
    model.add(LSTM(256, return_sequences=True))
    model.add(Dense(256, activation = 'relu'))
	
    model.add(Dropout(0.2))
    model.add(LSTM(32))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd)
    

    return model

def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "D:/Study/NEU/Neural Network and AI/final/Classical-Piano-Composer-master/Classical-Piano-Composer-master/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)

if __name__ == '__main__':
    train_network()


Using TensorFlow backend.


Parsing D:/Study/NEU/Neural Network and AI/final/Classical-Piano-Composer-master/Classical-Piano-Composer-master/midi_songs\0fithos.mid
Parsing D:/Study/NEU/Neural Network and AI/final/Classical-Piano-Composer-master/Classical-Piano-Composer-master/midi_songs\8.mid
Parsing D:/Study/NEU/Neural Network and AI/final/Classical-Piano-Composer-master/Classical-Piano-Composer-master/midi_songs\ahead_on_our_way_piano.mid
Parsing D:/Study/NEU/Neural Network and AI/final/Classical-Piano-Composer-master/Classical-Piano-Composer-master/midi_songs\AT.mid
Parsing D:/Study/NEU/Neural Network and AI/final/Classical-Piano-Composer-master/Classical-Piano-Composer-master/midi_songs\balamb.mid
Parsing D:/Study/NEU/Neural Network and AI/final/Classical-Piano-Composer-master/Classical-Piano-Composer-master/midi_songs\bcm.mid
Parsing D:/Study/NEU/Neural Network and AI/final/Classical-Piano-Composer-master/Classical-Piano-Composer-master/midi_songs\BlueStone_LastDungeon.mid
Parsing D:/Study/NEU/Neural Network

Parsing D:/Study/NEU/Neural Network and AI/final/Classical-Piano-Composer-master/Classical-Piano-Composer-master/midi_songs\JENOVA.mid
Parsing D:/Study/NEU/Neural Network and AI/final/Classical-Piano-Composer-master/Classical-Piano-Composer-master/midi_songs\Kingdom_Hearts_Dearly_Beloved.mid
Parsing D:/Study/NEU/Neural Network and AI/final/Classical-Piano-Composer-master/Classical-Piano-Composer-master/midi_songs\Kingdom_Hearts_Traverse_Town.mid
Parsing D:/Study/NEU/Neural Network and AI/final/Classical-Piano-Composer-master/Classical-Piano-Composer-master/midi_songs\Life_Stream.mid
Parsing D:/Study/NEU/Neural Network and AI/final/Classical-Piano-Composer-master/Classical-Piano-Composer-master/midi_songs\lurk_in_dark.mid
Parsing D:/Study/NEU/Neural Network and AI/final/Classical-Piano-Composer-master/Classical-Piano-Composer-master/midi_songs\mining.mid
Parsing D:/Study/NEU/Neural Network and AI/final/Classical-Piano-Composer-master/Classical-Piano-Composer-master/midi_songs\Oppressed.

57077/57077 [==============================] - 396s 7ms/step - loss: 4.1105
Epoch 41/200
57077/57077 [==============================] - 396s 7ms/step - loss: 4.0858
Epoch 42/200
57077/57077 [==============================] - 396s 7ms/step - loss: 4.0581
Epoch 43/200
57077/57077 [==============================] - 396s 7ms/step - loss: 4.0327
Epoch 44/200
57077/57077 [==============================] - 396s 7ms/step - loss: 4.0048
Epoch 45/200
57077/57077 [==============================] - 396s 7ms/step - loss: 3.9926
Epoch 46/200
57077/57077 [==============================] - 396s 7ms/step - loss: 3.9572
Epoch 47/200
57077/57077 [==============================] - 396s 7ms/step - loss: 3.9454
Epoch 48/200
57077/57077 [==============================] - 395s 7ms/step - loss: 3.8926
Epoch 49/200
57077/57077 [==============================] - 395s 7ms/step - loss: 3.8687
Epoch 50/200
57077/57077 [==============================] - 395s 7ms/step - loss: 3.8294
Epoch 51/200
57077/57077 [========

57077/57077 [==============================] - 461s 8ms/step - loss: 2.1263
Epoch 132/200
57077/57077 [==============================] - 460s 8ms/step - loss: 2.1133
Epoch 133/200
57077/57077 [==============================] - 461s 8ms/step - loss: 2.1106
Epoch 134/200
57077/57077 [==============================] - 459s 8ms/step - loss: 2.0957
Epoch 135/200
57077/57077 [==============================] - 461s 8ms/step - loss: 2.0817
Epoch 136/200
57077/57077 [==============================] - 456s 8ms/step - loss: 2.0679
Epoch 137/200
57077/57077 [==============================] - 449s 8ms/step - loss: 2.0526
Epoch 138/200
57077/57077 [==============================] - 396s 7ms/step - loss: 2.0278
Epoch 139/200
57077/57077 [==============================] - 396s 7ms/step - loss: 2.0388
Epoch 140/200
57077/57077 [==============================] - 396s 7ms/step - loss: 2.0205
Epoch 141/200
57077/57077 [==============================] - 395s 7ms/step - loss: 2.0049
Epoch 142/200
57077/5707